In [ ]:
from clinical_ts.timeseries_utils import *
from clinical_ts.ecg_utils import *
from pathlib import Path
import numpy as np

## PTB-XL

In [ ]:
target_fs=100 # sampling rate (100 Hz or 500 Hz)
data_folder_ptb_xl = Path("./data_folder_ptb_xl/")
target_folder_ptb_xl = Path("./processed_ptb_xl_fs"+str(target_fs)) 

In [ ]:
# Prepare the dataset
df_ptb_xl, lbl_itos_ptb_xl,  mean_ptb_xl, std_ptb_xl = prepare_data_ptb_xl(data_folder_ptb_xl, min_cnt=0, target_fs=target_fs, channels=12, channel_stoi=channel_stoi_default, target_folder=target_folder_ptb_xl)

In [ ]:
#reformat everything as memmap for efficiency
reformat_as_memmap(df_ptb_xl, target_folder_ptb_xl/("memmap.npy"),data_folder=target_folder_ptb_xl,delete_npys=True)

## PTB-XL dataloader

In [ ]:
input_size = 1000  # Sample length

chunkify_train = False
chunk_length_train = input_size if chunkify_train else 0
stride_train = input_size
        
chunkify_valtest = False
chunk_length_valtest = input_size if chunkify_valtest else 0
stride_valtest = input_size

In [ ]:
df_mapped, lbl_itos,  mean, std = load_dataset(target_folder_ptb_xl)

ds_mean = np.array([-0.00184586, -0.00130277,  0.00017031, -0.00091313, -0.00148835,  -0.00174687, -0.00077071, -0.00207407,  0.00054329,  0.00155546,  -0.00114379, -0.00035649])
ds_std = np.array([0.16401004, 0.1647168 , 0.23374124, 0.33767231, 0.33362807,  0.30583013, 0.2731171 , 0.27554379, 0.17128962, 0.14030828,   0.14606956, 0.14656108])

ptb_xl_label = "label_diag" # https://iphome.hhi.de/samek/pdf/WagSciData20.pdf 
                    
lbl_itos= np.array(lbl_itos[ptb_xl_label])
def multihot_encode(x, num_classes):
    res = np.zeros(num_classes,dtype=np.float32)
    for y in x:
        res[y]=1
    return res
df_mapped["label"]= df_mapped[ptb_xl_label+"_filtered_numeric"].apply(lambda x: multihot_encode(x,len(lbl_itos)))

tfms_ptb_xl_cpc = ToTensor()
            
max_fold_id = df_mapped.strat_fold.max() 
df_train = df_mapped[df_mapped.strat_fold<max_fold_id-1]
df_val = df_mapped[df_mapped.strat_fold==max_fold_id-1]
df_test = df_mapped[df_mapped.strat_fold==max_fold_id]

In [ ]:
# Here are the PTB-XL dataloaders

ds_train=TimeseriesDatasetCrops(df_train,input_size,num_classes=len(lbl_itos),data_folder=target_folder_ptb_xl,chunk_length=chunk_length_train,min_chunk_length=input_size, stride=stride_train,transforms=tfms_ptb_xl_cpc,annotation=False,col_lbl ="label" ,memmap_filename=target_folder_ptb_xl/("memmap.npy"))
ds_val=TimeseriesDatasetCrops(df_val,input_size,num_classes=len(lbl_itos),data_folder=target_folder_ptb_xl,chunk_length=chunk_length_valtest,min_chunk_length=input_size, stride=stride_valtest,transforms=tfms_ptb_xl_cpc,annotation=False,col_lbl ="label",memmap_filename=target_folder_ptb_xl/("memmap.npy"))
ds_test=TimeseriesDatasetCrops(df_test,input_size,num_classes=len(lbl_itos),data_folder=target_folder_ptb_xl,chunk_length=chunk_length_valtest,min_chunk_length=input_size, stride=stride_valtest,transforms=tfms_ptb_xl_cpc,annotation=False,col_lbl ="label",memmap_filename=target_folder_ptb_xl/("memmap.npy"))